In [1]:
import sys
!{sys.executable} -m pip install -U h2o

   ---------------------------------------- 0.0/266.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/266.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/266.0 MB 3.0 MB/s eta 0:01:28
   ---------------------------------------- 1.0/266.0 MB 2.8 MB/s eta 0:01:35
   ---------------------------------------- 1.6/266.0 MB 2.0 MB/s eta 0:02:10
   ---------------------------------------- 2.1/266.0 MB 2.2 MB/s eta 0:02:02
   ---------------------------------------- 2.9/266.0 MB 2.4 MB/s eta 0:01:49
    --------------------------------------- 3.4/266.0 MB 2.5 MB/s eta 0:01:47
    --------------------------------------- 4.2/266.0 MB 2.6 MB/s eta 0:01:39
    --------------------------------------- 4.2/266.0 MB 2.6 MB/s eta 0:01:39
    --------------------------------------- 5.0/266.0 MB 2.5 MB/s eta 0:01:46
    --------------------------------------- 5.8/266.0 MB 2.6 MB/s eta 0:01:42
    --------------------------------------- 6.3/266.0 MB 2.6 MB/s eta 0:01:40


In [2]:
import h2o
from h2o.automl import H2OAutoML

# 1) Start H2O
h2o.init(max_mem_size="4G")   # adjust if you want, e.g. "2G"

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 22.0.2+9-70, mixed mode, sharing)
  Starting server from E:\AutoML\evalml_env\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\DELL\AppData\Local\Temp\tmpxzguirku
  JVM stdout: C:\Users\DELL\AppData\Local\Temp\tmpxzguirku\h2o_DELL_started_from_python.out
  JVM stderr: C:\Users\DELL\AppData\Local\Temp\tmpxzguirku\h2o_DELL_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,10 secs
H2O_cluster_timezone:,Asia/Dhaka
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.9
H2O_cluster_version_age:,2 months and 26 days
H2O_cluster_name:,H2O_from_python_DELL_asvxh6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.978 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [3]:
# 2) Load CSV into H2OFrame
hf = h2o.import_file("ckd.csv")
hf.head()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


PatientID,Age,Gender,Ethnicity,SocioeconomicStatus,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,SleepQuality,FamilyHistoryKidneyDisease,FamilyHistoryHypertension,FamilyHistoryDiabetes,PreviousAcuteKidneyInjury,UrinaryTractInfections,SystolicBP,DiastolicBP,FastingBloodSugar,HbA1c,SerumCreatinine,BUNLevels,GFR,ProteinInUrine,ACR,SerumElectrolytesSodium,SerumElectrolytesPotassium,SerumElectrolytesCalcium,SerumElectrolytesPhosphorus,HemoglobinLevels,CholesterolTotal,CholesterolLDL,CholesterolHDL,CholesterolTriglycerides,ACEInhibitors,Diuretics,NSAIDsUse,Statins,AntidiabeticMedications,Edema,FatigueLevels,NauseaVomiting,MuscleCramps,Itching,QualityOfLifeScore,HeavyMetalsExposure,OccupationalExposureChemicals,WaterQuality,MedicalCheckupsFrequency,MedicationAdherence,HealthLiteracy,Diagnosis,DoctorInCharge
1,71,0,0,0,2,31.0694,1,5.12811,1.67622,0.240386,4.07643,0,0,0,0,0,113,83,72.5108,9.2124,4.96253,25.6059,45.7032,0.74498,123.849,137.653,3.62606,10.3144,3.15265,16.1147,207.729,85.8637,21.968,212.095,0,0,4.56314,1,0,0,3.56389,6.99224,4.51851,7.5563,76.0768,0,0,1,1.01882,4.96681,9.87145,1,Confidential
2,34,0,0,1,3,29.6921,1,18.6096,8.37757,6.50323,7.65281,1,1,0,0,0,120,67,100.849,4.60499,3.1568,31.3382,55.7845,3.05232,88.5391,138.141,5.33287,9.6042,2.85544,15.3492,189.451,86.3787,87.5698,255.451,0,0,9.097,0,0,0,5.32734,0.35629,2.20222,6.83677,40.1285,0,0,0,3.92354,8.18927,7.16177,1,Confidential
3,80,1,1,0,1,37.3948,1,11.8824,9.6074,2.10483,4.39279,0,0,0,0,0,147,106,160.989,5.4326,3.69824,39.7382,67.559,1.15784,21.1709,142.97,4.33089,9.88579,4.35351,13.0188,284.138,132.27,20.0498,251.903,0,1,3.85125,1,0,0,4.85542,4.67407,5.96727,2.14472,92.8728,0,1,1,1.42991,7.62403,7.35463,1,Confidential
4,40,0,2,0,1,31.3297,0,16.0202,0.408871,6.96442,6.28227,0,0,0,0,0,117,65,188.507,4.14447,2.86847,21.981,33.2025,3.74587,123.78,137.107,3.81074,9.99589,4.01613,15.0563,235.112,93.4437,58.2603,392.338,0,0,7.88176,0,0,0,8.53168,5.69145,2.17639,7.07719,90.0803,0,0,0,3.22642,3.28269,6.62959,1,Confidential
5,43,0,1,1,2,23.7263,0,7.94415,0.780319,3.0978,4.02164,0,0,0,0,0,98,66,82.1567,4.26298,3.96488,12.2164,56.3191,2.57099,184.852,140.628,4.86677,8.90762,3.94791,16.6906,258.278,171.758,21.5832,370.524,1,1,4.17946,1,0,0,1.42232,2.27346,6.80099,3.55312,5.25837,0,0,1,0.285466,3.8495,1.43738,1,Confidential
6,22,0,0,0,1,39.1556,0,4.24361,8.84725,8.73006,7.69136,0,0,0,0,0,134,79,141.924,4.17771,3.88433,41.0022,55.4769,1.09106,53.5493,137.005,5.09381,10.3693,2.58799,16.4793,211.26,102.019,48.2073,347.195,1,0,3.62707,0,1,1,6.97573,4.85546,3.34412,8.6857,12.7941,0,0,0,0.358613,5.7667,2.06694,1,Confidential
7,41,0,1,0,1,35.0405,0,18.2247,8.15552,3.36914,6.55249,0,1,0,0,0,179,76,160.419,8.80409,3.45716,19.9711,45.0304,2.46896,91.508,137.85,3.59475,9.55349,3.5454,10.6485,283.037,149.128,70.9412,184.144,0,0,9.46776,1,0,0,5.48887,1.31368,6.67273,1.99502,38.723,0,0,1,2.7446,5.51576,3.85668,1,Confidential
8,72,1,0,1,3,30.7604,1,18.6627,6.17934,6.19696,4.76212,0,0,0,0,0,138,95,105.93,5.78976,4.98984,17.9097,119.842,0.00440863,191.751,137.355,5.47439,8.6849,3.6509,13.7008,287.659,192.275,49.6961,100.785,1,1,6.99112,0,0,0,7.36825,0.895714,1.28519,2.17841,92.9398,0,0,0,1.82884,8.62047,9.95467,0,Confidential
9,21,0,1,0,2,22.3231,0,9.9515,9.44965,2.85928,5.77821,0,0,0,0,1,91,67,144.087,6.95927,2.56907,39.394,22.7309,4.3137,173.797,141.45,4.89405,9.29417,3.26504,13.1786,215.515,141.039,94.1513,257.153,0,0,8.40081,1,0,0,3.41946,6.47711,0.0797895,9.60586,84.4205,0,0,1,3.52197,4.72474,6.72574,1,Confidential
10,49,0,3,0,1,24.3385,0,0.1299,6.46431,9.17222,7.41796,0,0,0,0,0,95,119,162.982,8.72707,3.67787,47.0363,82.5099,0.0633919,260.636,140.267,4.1428,9.70329,3.14351,16.0161,186.109,86.323,71.2611,204.111,0,0,5.43358,0,0,0,4.7138,4.49775,1.37292,5.1702,4.77617,0,0,0,3.44888,8.93509,4.4238,1,Confidential


In [4]:
# 3) Drop columns
drop_cols = [c for c in ["PatientID", "DoctorInCharge"] if c in hf.columns]
if drop_cols:
    hf = hf.drop(drop_cols)

In [5]:
hf.head()

Age,Gender,Ethnicity,SocioeconomicStatus,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,SleepQuality,FamilyHistoryKidneyDisease,FamilyHistoryHypertension,FamilyHistoryDiabetes,PreviousAcuteKidneyInjury,UrinaryTractInfections,SystolicBP,DiastolicBP,FastingBloodSugar,HbA1c,SerumCreatinine,BUNLevels,GFR,ProteinInUrine,ACR,SerumElectrolytesSodium,SerumElectrolytesPotassium,SerumElectrolytesCalcium,SerumElectrolytesPhosphorus,HemoglobinLevels,CholesterolTotal,CholesterolLDL,CholesterolHDL,CholesterolTriglycerides,ACEInhibitors,Diuretics,NSAIDsUse,Statins,AntidiabeticMedications,Edema,FatigueLevels,NauseaVomiting,MuscleCramps,Itching,QualityOfLifeScore,HeavyMetalsExposure,OccupationalExposureChemicals,WaterQuality,MedicalCheckupsFrequency,MedicationAdherence,HealthLiteracy,Diagnosis
71,0,0,0,2,31.0694,1,5.12811,1.67622,0.240386,4.07643,0,0,0,0,0,113,83,72.5108,9.2124,4.96253,25.6059,45.7032,0.74498,123.849,137.653,3.62606,10.3144,3.15265,16.1147,207.729,85.8637,21.968,212.095,0,0,4.56314,1,0,0,3.56389,6.99224,4.51851,7.5563,76.0768,0,0,1,1.01882,4.96681,9.87145,1
34,0,0,1,3,29.6921,1,18.6096,8.37757,6.50323,7.65281,1,1,0,0,0,120,67,100.849,4.60499,3.1568,31.3382,55.7845,3.05232,88.5391,138.141,5.33287,9.6042,2.85544,15.3492,189.451,86.3787,87.5698,255.451,0,0,9.097,0,0,0,5.32734,0.35629,2.20222,6.83677,40.1285,0,0,0,3.92354,8.18927,7.16177,1
80,1,1,0,1,37.3948,1,11.8824,9.6074,2.10483,4.39279,0,0,0,0,0,147,106,160.989,5.4326,3.69824,39.7382,67.559,1.15784,21.1709,142.97,4.33089,9.88579,4.35351,13.0188,284.138,132.27,20.0498,251.903,0,1,3.85125,1,0,0,4.85542,4.67407,5.96727,2.14472,92.8728,0,1,1,1.42991,7.62403,7.35463,1
40,0,2,0,1,31.3297,0,16.0202,0.408871,6.96442,6.28227,0,0,0,0,0,117,65,188.507,4.14447,2.86847,21.981,33.2025,3.74587,123.78,137.107,3.81074,9.99589,4.01613,15.0563,235.112,93.4437,58.2603,392.338,0,0,7.88176,0,0,0,8.53168,5.69145,2.17639,7.07719,90.0803,0,0,0,3.22642,3.28269,6.62959,1
43,0,1,1,2,23.7263,0,7.94415,0.780319,3.0978,4.02164,0,0,0,0,0,98,66,82.1567,4.26298,3.96488,12.2164,56.3191,2.57099,184.852,140.628,4.86677,8.90762,3.94791,16.6906,258.278,171.758,21.5832,370.524,1,1,4.17946,1,0,0,1.42232,2.27346,6.80099,3.55312,5.25837,0,0,1,0.285466,3.8495,1.43738,1
22,0,0,0,1,39.1556,0,4.24361,8.84725,8.73006,7.69136,0,0,0,0,0,134,79,141.924,4.17771,3.88433,41.0022,55.4769,1.09106,53.5493,137.005,5.09381,10.3693,2.58799,16.4793,211.26,102.019,48.2073,347.195,1,0,3.62707,0,1,1,6.97573,4.85546,3.34412,8.6857,12.7941,0,0,0,0.358613,5.7667,2.06694,1
41,0,1,0,1,35.0405,0,18.2247,8.15552,3.36914,6.55249,0,1,0,0,0,179,76,160.419,8.80409,3.45716,19.9711,45.0304,2.46896,91.508,137.85,3.59475,9.55349,3.5454,10.6485,283.037,149.128,70.9412,184.144,0,0,9.46776,1,0,0,5.48887,1.31368,6.67273,1.99502,38.723,0,0,1,2.7446,5.51576,3.85668,1
72,1,0,1,3,30.7604,1,18.6627,6.17934,6.19696,4.76212,0,0,0,0,0,138,95,105.93,5.78976,4.98984,17.9097,119.842,0.00440863,191.751,137.355,5.47439,8.6849,3.6509,13.7008,287.659,192.275,49.6961,100.785,1,1,6.99112,0,0,0,7.36825,0.895714,1.28519,2.17841,92.9398,0,0,0,1.82884,8.62047,9.95467,0
21,0,1,0,2,22.3231,0,9.9515,9.44965,2.85928,5.77821,0,0,0,0,1,91,67,144.087,6.95927,2.56907,39.394,22.7309,4.3137,173.797,141.45,4.89405,9.29417,3.26504,13.1786,215.515,141.039,94.1513,257.153,0,0,8.40081,1,0,0,3.41946,6.47711,0.0797895,9.60586,84.4205,0,0,1,3.52197,4.72474,6.72574,1
49,0,3,0,1,24.3385,0,0.1299,6.46431,9.17222,7.41796,0,0,0,0,0,95,119,162.982,8.72707,3.67787,47.0363,82.5099,0.0633919,260.636,140.267,4.1428,9.70329,3.14351,16.0161,186.109,86.323,71.2611,204.111,0,0,5.43358,0,0,0,4.7138,4.49775,1.37292,5.1702,4.77617,0,0,0,3.44888,8.93509,4.4238,1


In [7]:
# 4) Set target + features
y = "Diagnosis"
x = [c for c in hf.columns if c != y]

In [8]:
# 5) Ensure target is categorical (classification)
hf[y] = hf[y].asfactor()

In [9]:
# 6) Train/Test split
train, test = hf.split_frame(ratios=[0.8], seed=42)

In [11]:
# 7) AutoML
aml = H2OAutoML(
    max_models=30,          # increase for better search
    seed=42,
    sort_metric="AUC"       # or "F1", "logloss", "accuracy"
)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
13:22:16.481: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),4/5
# GBM base models (used / total),1/1
# GLM base models (used / total),1/1
# DRF base models (used / total),1/2
# DeepLearning base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [12]:
# 8) Leaderboard
lb = aml.leaderboard
print(lb.head(rows=10))

model_id                                                      auc    logloss     aucpr    mean_per_class_error      rmse        mse
StackedEnsemble_BestOfFamily_1_AutoML_1_20260220_132216  0.827238   0.216468  0.967482                0.377048  0.238028  0.0566572
StackedEnsemble_AllModels_1_AutoML_1_20260220_132216     0.826895   0.208177  0.972041                0.347845  0.234367  0.0549279
GBM_3_AutoML_1_20260220_132216                           0.822573   0.221859  0.970137                0.366112  0.243858  0.0594666
GBM_5_AutoML_1_20260220_132216                           0.818591   0.239877  0.972321                0.4144    0.256203  0.0656401
GBM_grid_1_AutoML_1_20260220_132216_model_1              0.815209   0.239399  0.974317                0.447657  0.25493   0.0649891
GBM_1_AutoML_1_20260220_132216                           0.814675   0.218459  0.967418                0.372993  0.240884  0.0580253
GBM_grid_1_AutoML_1_20260220_132216_model_7              0.811162   0.226957

In [13]:
# 9) Best model
leader = aml.leader
print(leader)

Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_BestOfFamily_1_AutoML_1_20260220_132216


Model Summary for Stacked Ensemble: 
key                                        value
-----------------------------------------  ----------------
Stacking strategy                          cross_validation
Number of base models (used / total)       4/5
# GBM base models (used / total)           1/1
# GLM base models (used / total)           1/1
# DRF base models (used / total)           1/2
# DeepLearning base models (used / total)  1/1
Metalearner algorithm                      GLM
Metalearner fold assignment scheme         Random
Metalearner nfolds                         5
Metalearner fold_column
Custom metalearner hyperparameters         None

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.003165200368692979
RMSE: 0.0562601134792046
LogLoss: 0.031022813040025438
AUC: 1.0
AUCPR: 1.0
Gini: 1.0
Null degrees of freedom: 1332
R

In [14]:
# 10) Performance on test set
perf = leader.model_performance(test_data=test)
print(perf)

ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.04045413659794434
RMSE: 0.20113213715849673
LogLoss: 0.14426197979126484
AUC: 0.9294016358157554
AUCPR: 0.9937097802186106
Gini: 0.8588032716315108
Null degrees of freedom: 325
Residual degrees of freedom: 321
Null deviance: 167.11134736585672
Residual deviance: 94.05881082390468
AIC: 104.05881082390468

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6166675367579844
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      10   13   0.5652   (13.0/23.0)
1      3    300  0.0099   (3.0/303.0)
Total  13   313  0.0491   (16.0/326.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.616668     0.974026  312
max f2                       0.406135     0.987614  321
max f0point5                 0.879264     0.978187  290
max accuracy

In [15]:
# 11) Predictions on test
pred = leader.predict(test)
pred.head()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,p0,p1
1,0.00797283,0.992027
0,0.670904,0.329096
1,0.0151208,0.984879
1,0.0556931,0.944307
1,0.00674901,0.993251
1,0.00526487,0.994735
1,0.0109596,0.98904
1,0.011478,0.988522
1,0.058881,0.941119
1,0.0581805,0.941819


In [17]:
aml.leaderboard.head(rows=20)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_1_20260220_132216,0.827238,0.216468,0.967482,0.377048,0.238028,0.0566572
StackedEnsemble_AllModels_1_AutoML_1_20260220_132216,0.826895,0.208177,0.972041,0.347845,0.234367,0.0549279
GBM_3_AutoML_1_20260220_132216,0.822573,0.221859,0.970137,0.366112,0.243858,0.0594666
GBM_5_AutoML_1_20260220_132216,0.818591,0.239877,0.972321,0.4144,0.256203,0.0656401
GBM_grid_1_AutoML_1_20260220_132216_model_1,0.815209,0.239399,0.974317,0.447657,0.25493,0.0649891
GBM_1_AutoML_1_20260220_132216,0.814675,0.218459,0.967418,0.372993,0.240884,0.0580253
GBM_grid_1_AutoML_1_20260220_132216_model_7,0.811162,0.226957,0.965688,0.330397,0.247973,0.0614906
GBM_grid_1_AutoML_1_20260220_132216_model_6,0.80963,0.222161,0.96863,0.340964,0.24415,0.0596093
GBM_grid_1_AutoML_1_20260220_132216_model_9,0.803652,0.236201,0.972393,0.438729,0.253126,0.0640728
GBM_grid_1_AutoML_1_20260220_132216_model_8,0.803162,0.240014,0.970999,0.451712,0.252674,0.0638442


In [18]:
lb = aml.leaderboard.as_data_frame()

lb_individual = lb[~lb["model_id"].str.contains("StackedEnsemble")]

lb_individual_sorted = lb_individual.sort_values(by="auc", ascending=False)

lb_individual_sorted.head(5)

E:\AutoML\evalml_env\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
2,GBM_3_AutoML_1_20260220_132216,0.822573,0.221859,0.970137,0.366112,0.243858,0.059467
3,GBM_5_AutoML_1_20260220_132216,0.818591,0.239877,0.972321,0.414400,0.256203,0.065640
4,GBM_grid_1_AutoML_1_20260220_132216_model_1,0.815209,0.239399,0.974317,0.447657,0.254930,0.064989
5,GBM_1_AutoML_1_20260220_132216,0.814675,0.218459,0.967418,0.372993,0.240884,0.058025
6,GBM_grid_1_AutoML_1_20260220_132216_model_7,0.811162,0.226957,0.965688,0.330397,0.247973,0.061491


In [19]:
best_individual_id = lb_individual_sorted.iloc[0]["model_id"]
best_individual_model = h2o.get_model(best_individual_id)

print(best_individual_model)

Model Details
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_3_AutoML_1_20260220_132216


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    26                 26                          14306                  8            8            8             26            47            39.2308

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.01068186522213899
RMSE: 0.10335310939753574
LogLoss: 0.05745002980555628
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7984347637970519
       0    1     Error    Rate
-----  ---  ----  -------  ------------
0      112  0     0        (0.0/112.0)
1      0    1221  0        (0.0

In [20]:
perf_stack = aml.leader.model_performance(test_data=test)
perf_ind = best_individual_model.model_performance(test_data=test)

print("Stacked AUC:", perf_stack.auc())
print("Individual AUC:", perf_ind.auc())
print("Stacked F1:", perf_stack.F1())
print("Individual F1:", perf_ind.F1())

Stacked AUC: 0.9294016358157554
Individual AUC: 0.8892237049791936
Stacked F1: [[0.6166675367579844, 0.974025974025974]]
Individual F1: [[0.7936613659799423, 0.9786535303776683]]


In [21]:
perf_test = best_individual_model.model_performance(test_data=test)
print(perf_test)
print(perf_test.confusion_matrix())

ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.043783150898280596
RMSE: 0.20924423743147766
LogLoss: 0.15943512483283395
Mean Per-Class Error: 0.1821638685607691
AUC: 0.8892237049791936
AUCPR: 0.9865851745930875
Gini: 0.7784474099583871

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7936613659799423
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      15   8    0.3478   (8.0/23.0)
1      5    298  0.0165   (5.0/303.0)
Total  20   306  0.0399   (13.0/326.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.793661     0.978654  305
max f2                       0.511541     0.987614  321
max f0point5                 0.856533     0.976589  297
max accuracy                 0.793661     0.960123  305
max precision                0.992477     1         0
max recall                   0.51

In [22]:
thr_f1 = perf_test.find_threshold_by_max_metric("f1")
thr_acc = perf_test.find_threshold_by_max_metric("accuracy")
thr_tnr = perf_test.find_threshold_by_max_metric("tnr")  # specificity maximize

print("Best F1 threshold:", thr_f1)
print("Best accuracy threshold:", thr_acc)
print("Best specificity threshold:", thr_tnr)

print(perf_test.confusion_matrix(thresholds=[thr_tnr]))

Best F1 threshold: 0.7936613659799423
Best accuracy threshold: 0.7936613659799423
Best specificity threshold: 0.9924770868951661
Confusion Matrix (Act/Pred) @ threshold = 0.9924770868951661
       0    1    Error    Rate
-----  ---  ---  -------  -------------
0      23   0    0        (0.0/23.0)
1      302  1    0.9967   (302.0/303.0)
Total  325  1    0.9264   (302.0/326.0)


In [23]:
aml_bal = H2OAutoML(
    max_models=30,
    seed=42,
    sort_metric="AUC",
    balance_classes=True
)
aml_bal.train(x=x, y=y, training_frame=train)

print(aml_bal.leaderboard.head())

AutoML progress: |
14:07:09.231: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                      auc    logloss     aucpr    mean_per_class_error      rmse        mse
StackedEnsemble_BestOfFamily_1_AutoML_2_20260220_140709  0.81876    0.221749  0.967516                0.389621  0.241062  0.0581108
StackedEnsemble_AllModels_1_AutoML_2_20260220_140709     0.816738   0.215102  0.96811                 0.357593  0.238748  0.0570005
GBM_grid_1_AutoML_2_20260220_140709_model_7              0.81503    0.242934  0.969259                0.364883  0.260363  0.0677887
GBM_2_AutoML_2_20260220_140709                           0.814112   0.279035  0.972915                0.373033  0.271252  0.0735779
GBM_grid_1_AutoML_2_20260220_140709_model_6              0.808083   0.239859  0.967008                0.381512  0.256665  0.0658769
GLM_1_AutoML_2_20260220_140709                    